In [16]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [17]:
args = {
    'path': './dataset',
}

In [18]:
class SeqData(Dataset):
    def __init__(self, args, dataset='ABCD') -> None:
        super().__init__()
        root_path = args['path']
        datasets_path = list(dataset)
        self.datasets = list()
        for subset_path in datasets_path:
            with open(os.path.join(root_path, 'dataset' + subset_path)) as f:
                for data in f.readlines():
                    data = np.array(list(map(float, data.split())))
                    self.datasets.append(data)
        self.datasets = np.array(self.datasets)
        
    def __getitem__(self, index):
        data = self.datasets[index].astype('float32')
        cp, seq, label = data[0], data[1:-1], data[-1]
        return cp, seq, label

    def __len__(self):
        return np.shape(self.datasets)[0]

In [23]:
def data_analysis(dataset):
    set = SeqData(args, dataset)
    iter = DataLoader(set, batch_size=set.__len__(), shuffle=True)
    for cp, seq, label in iter:
        pass
    return

In [ ]:
data_analysis('A')